<a href="https://colab.research.google.com/github/Reape4er/python-for-bigdata/blob/main/%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import urllib.request
import re
import base64

url = "https://msk.spravker.ru/avtoservisy-avtotehcentry/"
with urllib.request.urlopen(url) as response:
    html = response.read().decode("utf-8")

org_blocks = re.findall(
    r'<div class="widgets-list__item">[\s\S]*?<div class="org-widget ">([\s\S]*?)</div>\s*</div>\s*</div>',
    html
)

with open("avtoservisy.csv", "w", encoding="utf-8") as f:
    f.write("Название,Адрес,Телефон,Часы работы,Сайт,Отзыв\n")

    for block in org_blocks:
        name_match = re.search(r'class="org-widget-header__title-link">([^<]+)</a>', block)
        name = name_match.group(1).strip() if name_match else ""

        addr_match = re.search(r'org-widget-header__meta--location">([^<]+)</span>', block)
        address = addr_match.group(1).strip() if addr_match else ""

        phone_match = re.search(r'<span[^>]*>Телефон</span>\s*</dt>\s*<dd[^>]*>([^<]+)</dd>', block, re.DOTALL)
        phone = phone_match.group(1).strip() if phone_match else ""

        hours_match = re.search(r'<span[^>]*>Часы работы</span>\s*</dt>\s*<dd[^>]*>([^<]+)</dd>', block, re.DOTALL)
        hours = hours_match.group(1).strip() if hours_match else ""

        site = ""
        data_url_match = re.search(r'data-url="([^"]+)"', block)
        if data_url_match:
            try:
                site = base64.b64decode(data_url_match.group(1)).decode("utf-8")
            except:
                site = ""

        review_match = re.search(r'<div class="org-widget-feedback__comment">\s*<p>([^<]+)', block)
        review = review_match.group(1).strip() if review_match else ""

        row = [
            name.replace(",", ";"),
            address.replace(",", ";"),
            phone.replace(",", ";"),
            hours.replace(",", ";"),
            site.replace(",", ";"),
            review.replace(",", ";")
        ]
        f.write('"' + '","'.join(row) + '"\n')